In [1]:
import openpyxl as pyxl
import pandas as pd
import numpy as np
from gspread.utils import a1_to_rowcol, rowcol_to_a1

import shutil, os

import re
from itertools import product
# we only need the function datetime.datetime.now; we can now reference it as dt.now
from datetime import datetime as dt

import sqlite3 as sql

def is_number(s) :
    try :
        float(s)
        return True
    except ValueError :
        return False

In [2]:
db_filename = 'my-budget-dev-v1.sqlite'

%run nuclear_option.py $db_filename
%run database_setup.py $db_filename

db = sql.connect(db_filename)

Creating an an empty database as "my-budget-dev-v1.sqlite"
Completed creating the database scheme. It looks as follows:


,name
0,money_pots
1,event_types
2,budget_pots
3,money_events
4,payments
5,recievings
6,transfers
7,budget_events
8,event_groups
9,event_in_group


The table money_pots: 


,key,description,liquid
0,KG,gemeinsames Konto,Yes
1,KE,Extrakonto zum gemeinsamen Konto,Yes
2,KM,Konto Max,Yes
3,KP,Konto Paul,Yes
4,KB,Konto Bundesbank,Yes
5,KC,Consorsbankkonto,No
6,BM,Bargeld Max,Yes
7,BP,Bargeld Paul,Yes
8,CB,Chipkarte Bundesbank,Semi
9,CT,Chipkarte Trianon,Semi


The table event_types: 


...

,group_id,event_id


The table database_event_types: 


,type,description
0,Erstellung,Erstellung eines Eintrages
1,Update,Hinzufügen von Information
2,Korrektur,Korrigieren eines Eintrages
3,Löschung,Löschen eines Eintrages


The table database_events: 


,id,type,date,description


In [3]:
excel_filename = 'GemeinsameBilanzierung_16_17__dev.xlsx'

current_sheet = 'August'
comment_sheet = 'august_'

# We take here the option data_only since for now we are not interested in the expression
wb = pyxl.load_workbook(excel_filename, data_only=True)


august = wb.get_sheet_by_name(current_sheet)
august_ = wb.copy_worksheet(august)
august_.title = comment_sheet
august_.sheet_state = 'hidden'

In [4]:
def list_from_range_string(range_string) :
    '''Extract all individual cell names from a excel range.
    
    Keyword arguments:
        range_string - The excel expression for the range
    
    Example:
        If range_string == 'A1:B3' then the list ['A1', 'B1', 'A2', 'B2', 'A3', 'B3'] is returned
    '''
    colon_position = range_string.find(':')
    if colon_position == -1 :
        raise
    first_cell = range_string[:colon_position]
    last_cell = range_string[colon_position+1:]
    
    first_row, first_col = a1_to_rowcol(first_cell)
    last_row, last_col = a1_to_rowcol(last_cell)
    
    return [rowcol_to_a1(i,j) for i,j in product(range(first_row, last_row+1), range(first_col, last_col+1))]

In [5]:
def generate_id(date) :
    '''Generate a new unique ID in the budgeter on the database. An ID is an integer with 8 digits, where 
    the first digit are based on the date and the last two digits are a serial number.
    
    Keyword arguments:
        date - the date on which the ID should be based
        
    Example: 
        If date==DateTime('2017-08-17') and the database contains the IDs 
        2017081701, 2017081702, 2017081703 and 2017081705, then the id 2017081704 is returned.
    
    Exceptions:
        IndexError - if all 99 possible serial numbers (01-99) have already been distributed.
        
        N.B. Could also Except, if the Database Call raises an exception.
    '''
    date_int = int(date.strftime('%Y%m%d00'))
    crsr = db.cursor()
    crsr.execute('SELECT id FROM money_events WHERE id BETWEEN {} AND {}'.format(date_int, date_int + 99))
    results = [row[0] for row in crsr.fetchall()]

    current_id = date_int + 1
    while current_id in results :
        current_id += 1
    if current_id > date_int + 99 :
        raise IndexError('Encountered to many ids for the date {}'.format(date))

    return current_id

In [6]:
def date_convert(item) :
    '''Try to get a DateTime from the excel cell, independent if it is represented by an integer, i.e. in 
    the native excel date format, or a string representation
    '''
    try :
          return pyxl.utils.datetime.from_excel(item)
    except :
        if type(item) is str :
            return pd.to_datetime(item)
        else :
            return pd.Timestamp(item)
        #pass
            

def get_df_by_range(sheet, first_cell, last_cell, date_cols=None) :
    '''Read a given range on the given sheet and return a DataFrame containing the data.
    
    Keyword arguments:
        sheet      - a openpyxl sheet object which is to be read
        first_cell - the top left cell of the range to be read; in excel cell notation
        last_cell  - the bottom right cell of the range to be read; in excel cell notation
        date_cols  - a column (or list of columns) which are assumed to contain dates and shall be returned
                     as pandas Timestamp object; can be given either as number (starting in zero) or a excel
                     column name
                     
        ToDo : Look if starting in zero is correct 
    '''
    data_rows = [[cell.value for cell in row] + ['{0}:{1}'.format(row[0].coordinate, row[-1].coordinate)]
        for row in august[first_cell:last_cell]]

    df = pd.DataFrame(data_rows)
    new_index = df.iloc[:,range(len(df.columns)-1)].dropna(how='all').index
    if date_cols is not None and type(date_cols) is int :
        df.iloc[:,date_cols] = df.iloc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is str : 
        df.loc[:,date_cols] = df.loc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is list :
        for col in date_cols :
            if type(col) is int :
                df.iloc[:,col] = df.iloc[:,col].apply(date_convert).copy()
            if type(col) is str : 
                df.loc[:,col] = df.loc[:,col].apply(date_convert).copy()
    return df.loc[new_index]

In [7]:
def put_comment_into_excel(sheet, cells, comment_text) :
    comment = pyxl.comments.Comment(comment_text, 'budgeter')
    if type(cells) is list :
        for cell in cells :
            sheet[cell].comment = comment
    elif type(cells) is str :
        sheet[cells].comment = comment

In [8]:
def put_payment_into_database_autogenerated(date, event_type, description, amount, 
                                            money_pot, budget_pot, comment) :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO payments VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id,  money_pot, amount))
    crsr.execute('''INSERT INTO budget_events VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id, budget_pot, amount))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

def put_transfer_into_database_autogenerated(date, event_type, description, amount, 
                                             money_pot_source, money_pot_sink, 
                                             effect_date = None, comment = '') :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO transfers VALUES ({}, "{}", "{}", {}, NULL, {});
        '''.format(the_id,  money_pot_source, money_pot_sink, amount,
                   'NULL' if effect_date is None else '"{}"'.format(effect_date)))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

In [9]:
def autogenerate_database_comment(filename, range_, comment_sheet) :
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {}. A note has been '.format(
               filename, range_) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet)

def autogenerate_database_comment_two_excel_ranges(filename, range1, range2, 
                                                   comment_sheet) :
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {} and {}. A note has been '.format(
               filename, range1, range2) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet)

def autogenerate_excel_comment(date, db_filename, the_id) :
    return 'On {} this cell was automatically read and '.format(date) + \
           'inserted into the database {}. The id ofthe entry is {}.'.format(
               date, db_filename, the_id)

In [10]:
budgeting = get_df_by_range(august_, 'A6', 'D130', 2)
budgeting.columns = ['budget_type', 'description', 'date', 'amount', 'excel_range']

col_titles = ['description', 'date', 'amount', 'excel_range']

max_bargeld = get_df_by_range(august_, 'H7', 'J130', 1)
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = get_df_by_range(august_, 'K7', 'M130', 1)
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = get_df_by_range(august_, 'N7', 'P130', 1)
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

In [11]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

all_info['treated'] = 'No'

In [23]:
display(all_info)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
0,R,Miete,2017-08-01,-568.000000,A6:D6,N13:P13,KG,both,Yes
1,R,Miete FFM,2017-08-02,-450.000000,A7:D7,N20:P20,KG,both,Yes
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.050000,A8:D8,N12:P12,KG,both,Yes
3,R,Strom EnviaM,2017-08-30,-51.000000,A9:D9,N89:P89,KG,both,Yes
4,R,Vodafone,2017-08-28,-19.990000,A10:D10,NaN,NaN,left_only,No
5,M,Telefonie,2017-08-28,-6.200000,A11:D11,NaN,NaN,left_only,No
6,R,Rechtsschutzversicherung,2017-01-30,-13.900000,A12:D12,NaN,NaN,left_only,No
7,R,Haftpflichtversicherung,2017-08-01,-7.500000,A13:D13,N11:P11,KG,both,Yes
8,G,GEW,2017-07-05,-2.500000,A14:D14,NaN,NaN,left_only,No
9,R,Semestergebühr Paul,2016-06-21,-46.151667,A15:D15,NaN,NaN,left_only,No


In [13]:
miete_info = all_info.loc[all_info.description == 'Miete'
                    ].loc[all_info.amount == -568
                    ].loc[all_info.treated != 'Yes']
for index, row in miete_info.iterrows() :
    database_comment = autogenerate_database_comment_two_excel_ranges(
        excel_filename, current_sheet + '!' + row['excel_range_x'],
        current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Dauerauftrag', 
                row['description'], row['amount'], row['money_pot'], 
                row['budget_type'], database_comment)
    
    crsr = db.cursor()
    crsr.execute('''INSERT INTO event_in_group VALUES (1, {});'''.format(the_id))
    db.commit()
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x']) + \
            list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [14]:
## Putting all the Info which had matching info on the left and the right hand of the budget sheet into
## the database.

## Only doing the "Barzahlung" payments right now

for BB in ['BM', 'BP'] : 
    full_match_bargeld = all_info[all_info['_merge'] == 'both'].loc[
                          all_info['money_pot'] == BB].loc[
                          all_info['treated'] == 'No']
    
    for index, row in full_match_bargeld.iterrows() :
        database_comment = autogenerate_database_comment_two_excel_ranges(
            excel_filename, current_sheet + '!' + row['excel_range_x'],
            current_sheet + '!' + row['excel_range_y'], comment_sheet)
        
        the_id = put_payment_into_database_autogenerated(row['date'], 'Barzahlung',
                     row['description'], row['amount'], row['money_pot'], 
                     row['budget_type'], database_comment)
       
        excel_comment = autogenerate_excel_comment(
            dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
        cells = list_from_range_string(row['excel_range_x']) + \
                list_from_range_string(row['excel_range_y'])
            
        put_comment_into_excel(august_, cells, excel_comment)
        
        all_info.loc[index, 'treated'] = 'Yes' 

In [15]:
for table in ["money_events", "payments", "budget_events"] :
    print('The table {}: '.format(table))
    display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))

The table money_events: 


,id,type,description,date
0,2017080101,Dauerauftrag,Miete,2017-08-01
1,2017080102,Barzahlung,Brot,2017-08-01
2,2017080103,Barzahlung,Brot,2017-08-01
3,2017080801,Barzahlung,Brot,2017-08-08
4,2017081301,Barzahlung,Eisessen,2017-08-13
5,2017081302,Barzahlung,Wasser,2017-08-13
6,2017081701,Barzahlung,Eisessen,2017-08-17
7,2017081801,Barzahlung,Toilette,2017-08-18
8,2017081901,Barzahlung,Spitzer,2017-08-19
9,2017082101,Barzahlung,PUCK aufladen,2017-08-21


The table payments: 


,id,money_pot,amount,additional_description,effect_date
0,2017080101,KG,-568.00,None,None
1,2017081301,BM,-4.80,None,None
2,2017081302,BM,-1.30,None,None
3,2017081701,BM,-1.20,None,None
4,2017081801,BM,-0.25,None,None
5,2017081901,BM,-2.00,None,None
6,2017082201,BM,-10.00,None,None
7,2017082301,BM,-14.00,None,None
8,2017082501,BM,-1.20,None,None
9,2017082801,BM,-1.20,None,None


The table budget_events: 


,id,budget_pot,amount,additional_description,budget_effet_date
0,2017080101,R,-568.00,None,None
1,2017081301,A,-4.80,None,None
2,2017081302,L,-1.30,None,None
3,2017081701,A,-1.20,None,None
4,2017081801,M,-0.25,None,None
5,2017081901,S,-2.00,None,None
6,2017082201,A,-10.00,None,None
7,2017082301,A,-14.00,None,None
8,2017082501,A,-1.20,None,None
9,2017082801,A,-1.20,None,None


In [16]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
full_match_konto = all_info[all_info['_merge'] == 'both'].loc[
                            all_info['money_pot'] == 'KG'].loc[
                            all_info['treated'] == 'No']
full_match_konto['payment_type'] = 'Kartenzahlung'
full_match_konto['use_data'] = 'X'

In [17]:
gk_wb_filename = 'testtest.xlsx'
full_match_konto.to_excel(gk_wb_filename)
gk_wb = pyxl.load_workbook(gk_wb_filename, data_only=True)

gk_sheet = gk_wb.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')
        
the_list_a = ['Barzahlung', 'Kartenzahlung', 'Überweisung', 'Dauerauftrag',
            'SEPA-Mandat', 'Bankeinzug', 'Abheben', 'Kontotransfer', 
            'Bargeldtransfer', 'Einnahme', 'Geldfund']
the_list_b = ['B', 'K', 'U', 'D', 'S', 'BE', 'A', 'KT', 'BT', 'E', 'GF']

the_list_str = '"' + ','.join(the_list_a + the_list_b) + '"'

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=the_list_str, allow_blank=False)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please select from the list'
dv1.promptTitle = 'List Selection'

dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X,x"', allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please check with an X or leave blank'
dv2.promptTitle = 'Checkmark'

dv1.ranges.append('K2:K' + str(gk_sheet.max_row))
dv2.ranges.append('L2:L' + str(gk_sheet.max_row))

gk_sheet.add_data_validation(dv1)
gk_sheet.add_data_validation(dv2)

for i in range(2, gk_sheet.max_row + 1) :
    gk_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    gk_sheet['K' + str(i)].fill = the_fill
    gk_sheet['L' + str(i)].fill = the_fill
    
for column_cells in gk_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    gk_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'H', 'I', 'J'] :
    gk_sheet.column_dimensions[row].hidden = True    
    
gk_wb.save(gk_wb_filename)

In [18]:
os.system('open ' + gk_wb_filename)
input('Make any input to continue.')

data_is_final = False

while not data_is_final :
    read_data = pd.read_excel(gk_wb_filename)
    
    ## Here one could do a replace with a dict if the transaction type was abbreviated

    print('The read data is:')
    display(read_data)
    print('')

    request = input('If this is not the intended data, enter any number:')

    if not is_number(request) :
        print('OK.')
        data_is_final = True
    else : 
        os.system('open ' + temp_name)
        input('Make any input to continue.')
        
checked_full_match_konto = read_data

Make any input to continue.
The read data is:


,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated,payment_type,use_data
1,R,Miete FFM,2017-08-02,-450.00,A7:D7,N20:P20,KG,both,No,Überweisung,X
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.05,A8:D8,N12:P12,KG,both,No,SEPA-Mandat,X
3,R,Strom EnviaM,2017-08-30,-51.00,A9:D9,N89:P89,KG,both,No,SEPA-Mandat,X
7,R,Haftpflichtversicherung,2017-08-01,-7.50,A13:D13,N11:P11,KG,both,No,SEPA-Mandat,X
12,R,Handy Max,2017-08-10,-7.99,A18:D18,N41:P41,KG,both,No,SEPA-Mandat,X
14,G,Spotify Max,2017-08-15,-4.99,A20:D20,N52:P52,KG,both,No,Kartenzahlung,X
15,G,Apple Music Paul,2017-08-25,-4.99,A21:D21,N76:P76,KG,both,No,Kartenzahlung,X
19,R,Monatskarte FFM,2017-07-31,-87.40,A25:D25,N9:P9,KG,both,No,Kartenzahlung,X
21,G,Fitnessstudio,2017-08-03,-39.80,A27:D27,N23:P23,KG,both,No,SEPA-Mandat,X
22,L,Aldi,2017-07-31,-23.97,A28:D28,N8:P8,KG,both,No,Kartenzahlung,X



If this is not the intended data, enter any number:
OK.


In [19]:
for index, row in checked_full_match_konto.iterrows() :
    if checked_full_match_konto.loc[index, 'use_data'] is np.nan :
        continue
    database_comment = autogenerate_database_comment_two_excel_ranges(
        excel_filename, current_sheet + '!' + row['excel_range_x'],
        current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], row['payment_type'],
                 row['description'], row['amount'], row['money_pot'], 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x']) + \
            list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [20]:
## List of all fully matched things we didnt treat yet
remaining_matched = all_info[all_info['_merge'] == 'both'].loc[
                            all_info['treated'] == 'No']

display(remaining_matched)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated


In [26]:
display(all_info[all_info['description'].str.contains('Schatulle')])

schatulle_info = all_info[all_info['treated'] == 'xxx']

for direction in ['an', 'aus'] :
    descriptor = 'Geld ' + direction + ' Schatulle'
    schatulle_info = schatulle_info.append(all_info[all_info['description'] == descriptor])
    
display(schatulle_info)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
122,NaN,Geld aus Schatulle,2017-08-05,1.30,NaN,H10:J10,BM,right_only,No
126,NaN,Geld aus Schatulle,2017-08-20,0.50,NaN,H19:J19,BM,right_only,No
127,NaN,Schatullenkorrektur,2017-08-20,1.05,NaN,H20:J20,BM,right_only,No
128,NaN,Schatullenkorrektur,2017-08-20,-1.05,NaN,H21:J21,BM,right_only,No
131,NaN,Geld aus Schatulle,NaT,1.00,NaN,K7:M7,BP,right_only,No
132,NaN,Geld aus Schatulle,2017-08-08,1.35,NaN,K10:M10,BP,right_only,No


,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
122,NaN,Geld aus Schatulle,2017-08-05,1.30,NaN,H10:J10,BM,right_only,No
126,NaN,Geld aus Schatulle,2017-08-20,0.50,NaN,H19:J19,BM,right_only,No
131,NaN,Geld aus Schatulle,NaT,1.00,NaN,K7:M7,BP,right_only,No
132,NaN,Geld aus Schatulle,2017-08-08,1.35,NaN,K10:M10,BP,right_only,No


In [28]:
for index, row in schatulle_info.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    i = 0
    while True :
        the_date = all_info.loc[index+i, 'date']
        if str(the_date) == 'NaT' :
            the_date = all_info.loc[index-i, 'date']
            if str(the_date) == 'NaT' :
                i = i + 1
                continue
        
        break
            
    the_id = put_transfer_into_database_autogenerated(
        the_date, 'Bargeldtransger', row['description'], row['amount'], 
        row['money_pot'], 'SB', database_comment)
    
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [34]:
## The Three individual usages of the White Trianon Card
trianon_aufladen = all_info[all_info['description'] == 'Geld aufladen Trianon'].loc[
                            all_info['amount'] == -20].loc[
                            all_info['treated'] == 'No']
for index, row in trianon_aufladen.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_transfer_into_database_autogenerated(row['date'], 'Aufladen',
                 row['description'], row['amount'], row['money_pot'], 'CT', 
                 database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'


trianon_doppelter_kaffee = all_info[all_info['budget_type'] == 'AM'].loc[
                            all_info['description'] == 'Doppelter Kaffee Trianon'].loc[
                            all_info['amount'] == -1.5].loc[
                            all_info['treated'] == 'No']
for index, row in trianon_doppelter_kaffee.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CT', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

trianon_einfacher_kaffee = all_info[all_info['budget_type'] == 'AM'].loc[
                            all_info['description'] == 'Kaffee'].loc[
                            all_info['amount'] == -0.75].loc[
                            all_info['treated'] == 'No']
for index, row in trianon_einfacher_kaffee.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CT', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'
    

In [30]:
bundesbank_karte = all_info[all_info['budget_type'] == 'AM'].loc[
                            all_info['treated'] == 'No']
for index, row in bundesbank_karte.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CB', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [31]:
geld_abheben = all_info[all_info['description'] == 'Geld abheben'].loc[
                        all_info['_merge'] == 'right_only'].loc[
                        all_info['treated'] == 'No']
geld_abheben_gotten = all_info[all_info['description'] == 'Geld abheben'].loc[
                              all_info['amount'] > 0].loc[
                              all_info['_merge'] == 'right_only'].loc[
                              all_info['treated'] == 'No']
geld_abheben_drawn = all_info[all_info['description'] == 'Geld abheben'].loc[
                              all_info['amount'] < 0].loc[
                              all_info['_merge'] == 'right_only'].loc[
                              all_info['treated'] == 'No']

geld_abheben_drawn['amount_abs'] = - geld_abheben_drawn['amount']

abheben_gotten = geld_abheben_gotten[['date', 'amount', 'money_pot', 'description', 
                                      'excel_range_y']].reset_index(level=0)
abheben_drawn = geld_abheben_drawn[['date', 'amount_abs', 'money_pot', 
                                    'excel_range_y']].reset_index(level=0)


abheben_data = pd.merge(abheben_gotten, abheben_drawn, how='outer', left_on = ['date', 'amount'], 
                        right_on=['date', 'amount_abs'], left_index=True, indicator=True)

display(abheben_data)


for index, row in abheben_data[abheben_data['_merge'] == 'both'].iterrows() :
    database_comment = autogenerate_database_comment_two_excel_ranges(
        excel_filename, current_sheet + '!' + row['excel_range_y_x'],
        current_sheet + '!' + row['excel_range_y_y'],comment_sheet)
            
    the_id = put_transfer_into_database_autogenerated(
        row['date'], 'Abheben', row['description'], row['amount'], 
        row['money_pot_y'], row['money_pot_x'], database_comment)
    
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y_x']) + \
            list_from_range_string(row['excel_range_y_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[row['index_x'], 'treated'] = 'Yes'
    all_info.loc[row['index_y'], 'treated'] = 'Yes'

,index_x,date,amount,money_pot_x,description,excel_range_y_x,index_y,amount_abs,money_pot_y,excel_range_y_y,_merge
1,120,2017-08-03,30.0,BM,Geld abheben,H8:J8,NaN,NaN,NaN,NaN,left_only
1,129,2017-08-22,40.0,BM,Geld abheben,H22:J22,146.0,40.0,KG,N68:P68,both
0,133,2017-08-21,20.0,BP,Geld abheben,K12:M12,145.0,20.0,KG,N65:P65,both


In [16]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
remaining = all_info[all_info['treated'] == 'No']
remaining['is_transfer'] = ''
remaining['money_pot_source'] = remaining['money_pot'] if remaining['amount'] < 0 else ''
remaining['money_pot_sink'] = remaining['money_pot'] if remaining['amount'] > 0 else ''

In [36]:
gk_wb_filename = 'testtest2.xlsx'
remaining.to_excel(gk_wb_filename)
gk_wb = pyxl.load_workbook(gk_wb_filename, data_only=True)

remaining_sheet = gk_wb.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')
        
#dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=the_list_str, allow_blank=False)
#dv1.error ='Your entry is not in the list'
#dv1.errorTitle = 'Invalid Entry'
#dv1.prompt = 'Please select from the list'
#dv1.promptTitle = 'List Selection'

dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X,x"', allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please check with an X or leave blank'
dv2.promptTitle = 'Checkmark'

#dv1.ranges.append('K2:K' + str(remaining_sheet.max_row))
dv2.ranges.append('K2:K' + str(remaining_sheet.max_row))

remaining_sheet.add_data_validation(dv1)
remaining_sheet.add_data_validation(dv2)

for i in range(2, remaining_sheet.max_row + 1) :
    remaining_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    remaining_sheet['K' + str(i)].fill = the_fill
    remaining_sheet['L' + str(i)].fill = the_fill
    
for column_cells in remaining_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    remaining_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'H', 'I', 'J'] :
    remaining_sheet.column_dimensions[row].hidden = True    
    
gk_wb.save(gk_wb_filename)

NameError: name 'remaining_sheet' is not defined

In [35]:
remaining = all_info[all_info['treated'] == 'No']

display(remaining)

print("number remaining: " + str(len(remaining)))

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
4,R,Vodafone,2017-08-28,-19.990000,A10:D10,NaN,NaN,left_only,No
5,M,Telefonie,2017-08-28,-6.200000,A11:D11,NaN,NaN,left_only,No
6,R,Rechtsschutzversicherung,2017-01-30,-13.900000,A12:D12,NaN,NaN,left_only,No
8,G,GEW,2017-07-05,-2.500000,A14:D14,NaN,NaN,left_only,No
9,R,Semestergebühr Paul,2016-06-21,-46.151667,A15:D15,NaN,NaN,left_only,No
10,R,Semestergebühr Max,2016-06-21,-50.565000,A16:D16,NaN,NaN,left_only,No
11,R,GEZ,2017-06-29,-17.500000,A17:D17,NaN,NaN,left_only,No
13,R,Handy Paul,2017-08-10,-7.990000,A19:D19,NaN,NaN,left_only,No
16,G,Backblaze Max,2017-04-18,-3.770833,A22:D22,NaN,NaN,left_only,No
17,R,Sommertickets,2017-07-20,-48.000000,A23:D23,NaN,NaN,left_only,No


number remaining: 41


In [ ]:
wb.save('GemBil.xlsx')

In [ ]:
db.close()